# 신경망 구현

* https://github.com/mnielsen/neural-networks-and-deep-learning/tree/master/src

In [10]:
import random

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

## MNIST Digit Image Test

In [11]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split

mnist = fetch_mldata('MNIST original')
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target)
X_train_new = [np.reshape(x, (784, 1)) for x in X_train]
y_train_new = [y[:, np.newaxis] for y in OneHotEncoder().fit_transform(y_train[:, np.newaxis]).toarray()]
X_test_new = [np.reshape(x, (784, 1)) for x in X_test]
training_data = zip(X_train_new, y_train_new)
test_data = zip(X_test_new, y_test)

## 패키지 사용방법

In [7]:
%cd /home/dockeruser/neural-networks-and-deep-learning/src
%ls

/home/dockeruser/neural-networks-and-deep-learning/src
conv.py                    mnist_loader.py   network2.py   network.py
expand_mnist.py            mnist_loader.pyc  network2.pyc  old/
mnist_average_darkness.py  mnist_svm.py      network3.py


In [8]:
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [10]:
import network

net = network.Network([784, 30, 10])
%time net.SGD(training_data, 30, 10, 0.1, test_data=test_data)

Epoch 0: 4767 / 10000
Epoch 1: 5620 / 10000
Epoch 2: 6272 / 10000
Epoch 3: 6531 / 10000
Epoch 4: 6667 / 10000
Epoch 5: 6779 / 10000
Epoch 6: 6839 / 10000
Epoch 7: 6911 / 10000
Epoch 8: 6959 / 10000
Epoch 9: 6999 / 10000
Epoch 10: 7026 / 10000
Epoch 11: 7054 / 10000
Epoch 12: 7080 / 10000
Epoch 13: 7103 / 10000
Epoch 14: 7112 / 10000
Epoch 15: 7128 / 10000
Epoch 16: 7772 / 10000
Epoch 17: 8213 / 10000
Epoch 18: 8240 / 10000
Epoch 19: 8258 / 10000
Epoch 20: 8270 / 10000
Epoch 21: 8290 / 10000
Epoch 22: 8301 / 10000
Epoch 23: 8314 / 10000
Epoch 24: 8335 / 10000
Epoch 25: 8795 / 10000
Epoch 26: 9061 / 10000
Epoch 27: 9092 / 10000
Epoch 28: 9116 / 10000
Epoch 29: 9128 / 10000
CPU times: user 2min 39s, sys: 0 ns, total: 2min 39s
Wall time: 2min 41s
